# 🎨 Data Designer Tutorial: Seeding Synthetic Data Generation with an External Dataset

#### 📚 What you'll learn

In this notebook, we will demonstrate how to seed synthetic data generation in Data Designer with an external dataset.

If this is your first time using Data Designer, we recommend starting with the [first notebook](https://nvidia-nemo.github.io/DataDesigner/latest/notebooks/1-the-basics/) in this tutorial series.


### 📦 Import the essentials

- The `essentials` module provides quick access to the most commonly used objects.


In [1]:
from data_designer.essentials import (
    ChatCompletionInferenceParams,
    DataDesigner,
    DataDesignerConfigBuilder,
    LocalFileSeedSource,
    ModelConfig,
)

### ⚙️ Initialize the Data Designer interface

- `DataDesigner` is the main object is responsible for managing the data generation process.

- When initialized without arguments, the [default model providers](https://nvidia-nemo.github.io/DataDesigner/latest/concepts/models/default-model-settings/) are used.


In [2]:
data_designer = DataDesigner()

### 🎛️ Define model configurations

- Each `ModelConfig` defines a model that can be used during the generation process.

- The "model alias" is used to reference the model in the Data Designer config (as we will see below).

- The "model provider" is the external service that hosts the model (see the [model config](https://nvidia-nemo.github.io/DataDesigner/latest/concepts/models/default-model-settings/) docs for more details).

- By default, we use [build.nvidia.com](https://build.nvidia.com/models) as the model provider.


In [3]:
# This name is set in the model provider configuration.
MODEL_PROVIDER = "nvidia"

# The model ID is from build.nvidia.com.
MODEL_ID = "nvidia/nemotron-3-nano-30b-a3b"

# We choose this alias to be descriptive for our use case.
MODEL_ALIAS = "nemotron-nano-v3"

model_configs = [
    ModelConfig(
        alias=MODEL_ALIAS,
        model=MODEL_ID,
        provider=MODEL_PROVIDER,
        inference_parameters=ChatCompletionInferenceParams(
            temperature=1.0,
            top_p=1.0,
            max_tokens=2048,
            extra_body={"chat_template_kwargs": {"enable_thinking": False}},
        ),
    )
]

### 🏗️ Initialize the Data Designer Config Builder

- The Data Designer config defines the dataset schema and generation process.

- The config builder provides an intuitive interface for building this configuration.

- The list of model configs is provided to the builder at initialization.


In [4]:
config_builder = DataDesignerConfigBuilder(model_configs=model_configs)

## 🏥 Prepare a seed dataset

- For this notebook, we'll create a synthetic dataset of patient notes.

- We will _seed_ the generation process with a [symptom-to-diagnosis dataset](https://huggingface.co/datasets/gretelai/symptom_to_diagnosis).

- We already have the dataset downloaded in the [data](../data) directory of this repository.

<br>

> 🌱 **Why use a seed dataset?**
>
> - Seed datasets let you steer the generation process by providing context that is specific to your use case.
>
> - Seed datasets are also an excellent way to inject real-world diversity into your synthetic data.
>
> - During generation, prompt templates can reference any of the seed dataset fields.


In [5]:
# Download sample dataset from Github
import urllib.request

url = "https://raw.githubusercontent.com/NVIDIA/GenerativeAIExamples/refs/heads/main/nemo/NeMo-Data-Designer/data/gretelai_symptom_to_diagnosis.csv"
local_filename, _ = urllib.request.urlretrieve(url, "gretelai_symptom_to_diagnosis.csv")

# Seed datasets are passed as reference objects to the config builder.
seed_source = LocalFileSeedSource(path=local_filename)

config_builder.with_seed_dataset(seed_source)

DataDesignerConfigBuilder()

## 🎨 Designing our synthetic patient notes dataset

- Here we use `add_column` with keyword arguments (rather than imported config objects).

- Generally, we recommend using concrete objects, but this is a convenient shorthand.

- **Note**: The prompt template can reference fields from our seed dataset:
  - `{{ diagnosis }}` - the medical diagnosis from the seed data
  - `{{ patient_summary }}` - the symptom description from the seed data


In [6]:
config_builder.add_column(
    name="patient_sampler",
    column_type="sampler",
    sampler_type="person_from_faker",
)

config_builder.add_column(
    name="doctor_sampler",
    column_type="sampler",
    sampler_type="person_from_faker",
)

config_builder.add_column(
    name="patient_id",
    column_type="sampler",
    sampler_type="uuid",
    params={
        "prefix": "PT-",
        "short_form": True,
        "uppercase": True,
    },
)

config_builder.add_column(
    name="first_name",
    column_type="expression",
    expr="{{ patient_sampler.first_name}}",
)

config_builder.add_column(
    name="last_name",
    column_type="expression",
    expr="{{ patient_sampler.last_name }}",
)


config_builder.add_column(
    name="dob",
    column_type="expression",
    expr="{{ patient_sampler.birth_date }}",
)

config_builder.add_column(
    name="symptom_onset_date",
    column_type="sampler",
    sampler_type="datetime",
    params={"start": "2024-01-01", "end": "2024-12-31"},
)

config_builder.add_column(
    name="date_of_visit",
    column_type="sampler",
    sampler_type="timedelta",
    params={"dt_min": 1, "dt_max": 30, "reference_column_name": "symptom_onset_date"},
)

config_builder.add_column(
    name="physician",
    column_type="expression",
    expr="Dr. {{ doctor_sampler.last_name }}",
)

config_builder.add_column(
    name="physician_notes",
    column_type="llm-text",
    prompt="""\
You are a primary-care physician who just had an appointment with {{ first_name }} {{ last_name }},
who has been struggling with symptoms from {{ diagnosis }} since {{ symptom_onset_date }}.
The date of today's visit is {{ date_of_visit }}.

{{ patient_summary }}

Write careful notes about your visit with {{ first_name }},
as Dr. {{ doctor_sampler.first_name }} {{ doctor_sampler.last_name }}.

Format the notes as a busy doctor might.
Respond with only the notes, no other text.
""",
    model_alias=MODEL_ALIAS,
)

data_designer.validate(config_builder)

[22:54:56] [INFO] ✅ Validation passed


### 🔁 Iteration is key – preview the dataset!

1. Use the `preview` method to generate a sample of records quickly.

2. Inspect the results for quality and format issues.

3. Adjust column configurations, prompts, or parameters as needed.

4. Re-run the preview until satisfied.


In [7]:
preview = data_designer.preview(config_builder, num_records=2)

[22:54:56] [INFO] 👁️ Preview generation in progress


[22:54:56] [INFO] ✅ Validation passed


[22:54:56] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph


[22:54:56] [INFO] 🩺 Running health checks for models...


[22:54:56] [INFO]   |-- 👀 Checking 'nvidia/nemotron-3-nano-30b-a3b' in provider named 'nvidia' for model alias 'nemotron-nano-v3'...


...

[22:55:01] [INFO]   |-- 🎲 column: 'symptom_onset_date'


[22:55:01] [INFO]   |-- 🎲 column: 'date_of_visit'


[22:55:01] [INFO]   |-- 🧩 column: 'physician'


[22:55:01] [INFO]   |-- 📝 column: 'physician_notes'


[22:55:01] [INFO] ☀️ Preview complete!


In [8]:
# Run this cell multiple times to cycle through the 2 preview records.
preview.display_sample_record()

                                                 Generated Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name               ┃ Value                                                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler    │ {                                                                                          │
│                    │     'uuid': 'adac8dbb-fb14-4606-a7e9-ff0737e14925',                                        │
│                    │     'locale': 'en_US',                                                                     │
│                    │     'first_name': 'Robin',                                                                 │
│                    │     'last_name': 'Maldonado',                                                              │
│                    │     'middle_name': None,                                                                   │
│                    │     'sex': 'Female',                                                                       │
│                    │     'street_number': '04834',                                                              │
│                    │     'street_name': 'Jodi Tunnel',                                                          │
│                    │     'city': 'Clayport',                                                                    │
│                    │     'state': 'Mississippi',                                                                │
│                    │     'postcode': '75491',                                                                   │
│                    │     'age': 81,                                                                             │
│                    │     'birth_date': '1944-12-03',                                                            │
│                    │     'country': 'Mexico',                                                                   │
│                    │     'marital_status': 'divorced',                                                          │
│                    │     'education_level': 'bachelors',                                                        │
│                    │     'unit': '',                                                                            │
│                    │     'occupation': 'Learning disability nurse',                                             │
│                    │     'phone_number': '(989)225-7338',                                                       │
│                    │     'bachelors_field': 'arts_humanities'                                                   │
│                    │ }                                                                                          │
├────────────────────┼────────────────────────────────────────────────────────────────────────────────────────────┤
│ doctor_sampler     │ {                                                                                          │
│                    │     'uuid': '0948b3a8-c220-487c-9828-4a6a8908931d',                                        │
│                    │     'locale': 'en_US',                                                                     │
│                    │     'first_name': 'Cynthia',                                                               │
│                    │     'last_name': 'Calderon',                                                               │
│                    │     'middle_name': None,                                                                   │
│                    │     'sex': 'Female',                                                                       │
│                    │     'street_number': '0742',     

In [9]:
# The preview dataset is available as a pandas DataFrame.
preview.dataset

,diagnosis,patient_summary,patient_sampler,doctor_sampler,patient_id,symptom_onset_date,date_of_visit,first_name,last_name,dob,physician,physician_notes
0,cervical spondylosis,I've been having a lot of pain in my neck and ...,{'uuid': 'adac8dbb-fb14-4606-a7e9-ff0737e14925...,{'uuid': '0948b3a8-c220-487c-9828-4a6a8908931d...,PT-AA950897,2024-07-25,2024-08-20,Robin,Maldonado,1944-12-03,Dr. Calderon,**SOAP Note** \n**Patient:** Robin Maldonado ...
1,impetigo,I have a rash on my face that is getting worse...,{'uuid': 'd28f73c5-0ae2-44dc-a395-56868336b3fe...,{'uuid': 'e0994940-9380-4b63-af68-5d586327f9f7...,PT-4B5C47EC,2024-02-04,2024-02-26,Melanie,Holmes,1966-03-30,Dr. Simmons,**Patient:** Melanie Holmes \n**DOB:** [redac...


### 📊 Analyze the generated data

- Data Designer automatically generates a basic statistical analysis of the generated data.

- This analysis is available via the `analysis` property of generation result objects.


In [10]:
# Print the analysis as a table.
preview.analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 2                               │ 10                              │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                   ┃       data type ┃             number unique values ┃               sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler               │            dict │                       2 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ doctor_sampler                │            dict │                       2 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ patient_id                    │          string │                       2 (100.0%) │                       uuid │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ symptom_onset_date            │          string │                       2 (100.0%) │                   datetime │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ date_of_visit                 │          string │                       2 (100.0%) │                  timedelta │
└───────────────────────────────┴─────────────────┴──────────────────────────────────┴────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                📝 LLM-Text Columns                                                
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                       ┃               ┃                            ┃     prompt tokens ┃      completion tokens ┃
┃ column name           ┃     data type ┃       number unique values ┃        per record ┃             per record ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ physician_notes       │        string │                 2 (100.0%) │     124.0 +/- 6.0 │        817.0 +/- 363.4 │
└───────────────────────┴───────────────┴────────────────────────────┴───────────────────┴────────────────────────┘
                                                                                                                   
                                                          

### 🆙 Scale up!

- Happy with your preview data?

- Use the `create` method to submit larger Data Designer generation jobs.


In [11]:
results = data_designer.create(config_builder, num_records=10, dataset_name="tutorial-3")

[22:55:01] [INFO] 🎨 Creating Data Designer dataset


[22:55:01] [INFO] ✅ Validation passed


[22:55:01] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph


[22:55:01] [INFO] 🩺 Running health checks for models...


[22:55:01] [INFO]   |-- 👀 Checking 'nvidia/nemotron-3-nano-30b-a3b' in provider named 'nvidia' for model alias 'nemotron-nano-v3'...


...

[22:55:14] [INFO]   |-- 🧩 column: 'dob'


[22:55:14] [INFO]   |-- 🎲 column: 'symptom_onset_date'


[22:55:14] [INFO]   |-- 🎲 column: 'date_of_visit'


[22:55:14] [INFO]   |-- 🧩 column: 'physician'


[22:55:14] [INFO]   |-- 📝 column: 'physician_notes'


In [12]:
# Load the generated dataset as a pandas DataFrame.
dataset = results.load_dataset()

dataset.head()

,diagnosis,patient_summary,patient_sampler,doctor_sampler,patient_id,symptom_onset_date,date_of_visit,first_name,last_name,dob,physician,physician_notes
0,cervical spondylosis,I've been having a lot of pain in my neck and ...,"{'age': 71, 'bachelors_field': 'arts_humanitie...","{'age': 44, 'bachelors_field': 'no_degree', 'b...",PT-7A0FE597,2024-06-04,2024-06-08,Paula,Hughes,1954-11-04,Dr. Jimenez,Date: 2024-06-08 Patient: Paula Hughes DOB...
1,impetigo,I have a rash on my face that is getting worse...,"{'age': 38, 'bachelors_field': 'no_degree', 'b...","{'age': 19, 'bachelors_field': 'no_degree', 'b...",PT-12D0B732,2024-12-27,2024-12-30,Melissa,Miller,1987-02-25,Dr. Costa,**Patient:** Melissa Miller **DOB:** [REDACT...
2,urinary tract infection,I have been urinating blood. I sometimes feel ...,"{'age': 46, 'bachelors_field': 'education', 'b...","{'age': 91, 'bachelors_field': 'education', 'b...",PT-AD0A0A7B,2024-03-21,2024-03-26,Karen,Cooper,1979-04-02,Dr. Davis,"**Progress Notes - Dr. Tina Davis, MD** **Da..."
3,arthritis,I have been having trouble with my muscles and...,"{'age': 76, 'bachelors_field': 'no_degree', 'b...","{'age': 27, 'bachelors_field': 'no_degree', 'b...",PT-DF12BEE7,2024-08-07,2024-08-28,Annette,Chan,1949-07-23,Dr. Rowland,**Date: 2024-08-28** **Patient:** Annette Ch...
4,dengue,I have been feeling really sick. My body hurts...,"{'age': 84, 'bachelors_field': 'no_degree', 'b...","{'age': 28, 'bachelors_field': 'no_degree', 'b...",PT-13581E03,2024-07-12,2024-08-04,Rebecca,Solomon,1941-04-04,Dr. Johnson,**RETURN TO CLINIC: 08/04/2024 | 09:45 AM** -...


In [13]:
# Load the analysis results into memory.
analysis = results.load_analysis()

analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 10                              │ 10                              │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                   ┃       data type ┃             number unique values ┃               sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler               │            dict │                      10 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ doctor_sampler                │            dict │                      10 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ patient_id                    │          string │                      10 (100.0%) │                       uuid │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ symptom_onset_date            │          string │                      10 (100.0%) │                   datetime │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ date_of_visit                 │          string │                        9 (90.0%) │                  timedelta │
└───────────────────────────────┴─────────────────┴──────────────────────────────────┴────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                📝 LLM-Text Columns                                                
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                       ┃               ┃                            ┃     prompt tokens ┃      completion tokens ┃
┃ column name           ┃     data type ┃       number unique values ┃        per record ┃             per record ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ physician_notes       │        string │                10 (100.0%) │     118.0 +/- 6.2 │        804.5 +/- 412.0 │
└───────────────────────┴───────────────┴────────────────────────────┴───────────────────┴────────────────────────┘
                                                                                                                   
                                                          

## ⏭️ Next Steps

Check out the following notebook to learn more about:

- [Providing images as context](https://nvidia-nemo.github.io/DataDesigner/latest/notebooks/4-providing-images-as-context/)
